1 

1.1

In [2]:
import sys
sys.path.append(r"C:/Users/Alexandre/Documents/GitHub/si/src")
from si.io.csv_file import read_csv
from si.data.dataset import Dataset
import numpy as np
import pandas as pd

dataset_iris = read_csv(r"C:/Users/Alexandre/Documents/GitHub/si/datasets/iris/iris.csv", features=True, label=True )
dataset_iris.shape()

(150, 4)

1.2

In [3]:
dataset_iris.features[-2]

'petal_length'

In [4]:
penultima_var=dataset_iris.X[:,-2]
penultima_var.shape

(150,)

1.3

In [5]:
last_ten = dataset_iris.X[-10:,:]
for i in range(len(dataset_iris.features)):
    array=last_ten[:,i]
    print(f'{dataset_iris.features[i]}: {round(array.mean(),3)}')

sepal_length: 6.45
sepal_width: 3.03
petal_length: 5.33
petal_width: 2.17


1.4

In [28]:
samples=[]
for line in dataset_iris.X:
   if np.all(line<=6):
      samples.append(line)
len(samples)

89

1.5

In [30]:
non_iris_setosa_samples=[]
for line in dataset_iris.y:
    if line != 'Iris-setosa':
        non_iris_setosa_samples.append(line)
len(non_iris_setosa_samples)

100

2

2.3

In [35]:
from si.data.dataset import Dataset
X = np.array([
            [1, np.nan, 3],
            [4, 5, np.nan]
        ])
dataset = Dataset(X, y=None)

dataset.fillna(value=0)
dataset.X

array([[1., 0., 3.],
       [4., 5., 0.]])

In [36]:
X = np.array([
            [1, 2, np.nan],
            [4, 5, 6],
            [np.nan, 1, 2]
            ])
y = np.array([1, 2, 3])
dataset = Dataset(X, y)

dataset.dropna()
dataset.X

array([[4., 5., 6.]])

In [38]:
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([10, 20, 30])
dataset = Dataset(X, y)

dataset.remove_by_index(1)

dataset.X

array([[1, 2],
       [5, 6]])

3

3.3

In [8]:
from si.feature_selection.select_percentile import SelectPercentile
from si.statistics.f_classification import f_classification

# percentil=25, para escolher a feature, com maior score no f_classification(oneway ANOVA)
# 4 features -> 100% / 4 = 25%
selector=SelectPercentile(score_func=f_classification, percentile=25)
selector.fit(dataset_iris)
new_dataset=selector.transform(dataset_iris)
new_dataset.features

Index(['petal_length'], dtype='object')

In [9]:
print('F-values:',selector.F)
print('p-values:', selector.p)

F-values: [ 119.26450218   47.3644614  1179.0343277   959.32440573]
p-values: [1.66966919e-31 1.32791652e-16 3.05197580e-91 4.37695696e-85]


4

In [10]:
from si.statistics.tanimoto_similarity import tanimoto_similarity
x = [1,0,1,1]
y = [
    [1,1,0,1],
    [0,0,1,1],
    [1,0,1,0]
]

tanimoto_similarity(x,y)

[0.5, 0.6666666666666666, 0.6666666666666666]

5

5.2

In [11]:
from si.decomposition.pca import PCA
pca=PCA(n_components=2)
pca.fit(dataset_iris)
pca.transform(dataset_iris)
pca.explained_variance

array([0.92461621, 0.05301557])

In [12]:
from sklearn.decomposition import PCA as skpca

sk_pca=skpca(n_components=2)
sk_pca.fit(dataset_iris.X)
sk_pca.transform(dataset_iris.X)
sk_pca.explained_variance_/sk_pca.explained_variance_.sum()

array([0.94577144, 0.05422856])

In [13]:
print(pca.components)
print(sk_pca.components_)

[[ 0.36158968 -0.08226889  0.85657211  0.35884393]
 [-0.65653988 -0.72971237  0.1757674   0.07470647]]
[[ 0.36158968 -0.08226889  0.85657211  0.35884393]
 [ 0.65653988  0.72971237 -0.1757674  -0.07470647]]


6

6.2

In [14]:
from si.model_selection.split import stratified_train_test_split
train,test=stratified_train_test_split(dataset_iris, 0.2, 42)

In [15]:
print(train.shape())
test.shape()

(120, 4)


(30, 4)

In [16]:
# proporções no dataset original
unique, counts = np.unique(dataset_iris.y, return_counts=True)
print("Original:", counts / counts.sum())

# proporções no treino
u_train, c_train = np.unique(train.y, return_counts=True)
print("Train:", c_train / c_train.sum())

# proporções no teste
u_test, c_test = np.unique(test.y, return_counts=True)
print("Test:", c_test / c_test.sum())

print("Classes treino:", np.unique(train.y))
print("Classes teste:", np.unique(test.y))


Original: [0.33333333 0.33333333 0.33333333]
Train: [0.33333333 0.33333333 0.33333333]
Test: [0.33333333 0.33333333 0.33333333]
Classes treino: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
Classes teste: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


7

7.3

In [17]:
from si.models.knn_regressor import KNNregressor
dataset_cpu = read_csv(r"C:/Users/Alexandre/Documents/GitHub/si/datasets/cpu/cpu.csv", features=True, label=True )
dataset_cpu.shape()

(209, 6)

In [18]:
knnregressor=KNNregressor(k=20)
knnregressor.fit(dataset_cpu)
y_pred=knnregressor.predict(dataset_cpu)
print('Preds:',y_pred[:20])
print('Score RMSE:',knnregressor.score(dataset_cpu))

Preds: [ 40.85 274.95 274.95 274.95 124.95 284.65 330.3  330.3  476.65 476.65
  59.7   29.9   65.65  98.65  20.05  84.35  53.2   39.6   52.3   44.9 ]
Score RMSE: 94.636


In [19]:
from sklearn.neighbors import KNeighborsRegressor

sk_knnregressor = KNeighborsRegressor(n_neighbors=20)

# treino
sk_knnregressor.fit(dataset_cpu.X, dataset_cpu.y)

# predição (dataset completo)
sk_pred = sk_knnregressor.predict(dataset_cpu.X)
print('Preds:',sk_pred[:20])
print('Score:',sk_knnregressor.score(dataset_cpu.X,dataset_cpu.y))

Preds: [ 40.85 274.95 274.95 274.95 124.95 284.65 330.3  330.3  476.65 476.65
  59.7   29.9   65.65  99.05  20.05  84.35  53.2   39.6   52.3   44.9 ]
Score: 0.6516698568973874


In [20]:
# Usando a sua função RMSE
# O dataset.y é y_true
# O sk_pred é y_pred
my_rmse = knnregressor._score(dataset_cpu, sk_pred) 
print('Sklearn RMSE:', my_rmse) 
# Este valor é MUITO próximo do 94.636 (a diferença é mínima devido a arredondamentos)

Sklearn RMSE: 94.694


8

In [21]:
from si.models.ridge_regression_least_squares import RidgeRegressionLeastSquares

rrls= RidgeRegressionLeastSquares(l2_penalty=1.0, scale=True)
rrls.fit(dataset_cpu)
print("Theta zero (intercept):", rrls.theta_zero)
print("Theta:", rrls.theta)


Theta zero (intercept): 105.61722488038275
Theta: [12.48809072 58.902124   64.91145823 25.9868963  -1.48080797 38.23854302]


In [22]:
pred_y = rrls.predict(dataset_cpu)
pred_y[:10]   
print("MSE:", rrls.score(dataset_cpu))


MSE: 3478.6907903184483


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

# Escalar dados para o sklearn
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset_cpu.X)

# Treinar Ridge com dados escalados
ridge_scaled = Ridge(alpha=1.0)
ridge_scaled.fit(X_scaled, dataset_cpu.y)

print("Sklearn Intercept:", ridge_scaled.intercept_)
print("Sklearn Scaled Coefs:", ridge_scaled.coef_)

Sklearn Intercept: 105.61722488038278
Sklearn Scaled Coefs: [12.48809072 58.902124   64.91145823 25.9868963  -1.48080797 38.23854302]


In [24]:
from sklearn.linear_model import Ridge

ridge_sklearn = Ridge(alpha=1.0)
ridge_sklearn.fit(dataset_cpu.X, dataset_cpu.y)

print("My model:", pred_y[:5])
print("Sklearn:", ridge_sklearn.predict(dataset_cpu.X)[:5])


My model: [337.20662835 311.31898441 311.31898441 311.31898441 198.94927755]
Sklearn: [337.16002677 311.94414755 311.94414755 311.94414755 199.08377291]


In [25]:
print("My model:", pred_y[:5])
print("Sklearn:", ridge_scaled.predict(X_scaled)[:5])

My model: [337.20662835 311.31898441 311.31898441 311.31898441 198.94927755]
Sklearn: [337.20662835 311.31898441 311.31898441 311.31898441 198.94927755]


9

9.2

In [26]:
from si.models.random_forest_classifier import RandomForestClassifier
from si.model_selection.split import train_test_split

train_set, test_set = train_test_split(dataset_iris, test_size=0.33, random_state=42)

model_rf = RandomForestClassifier(
    n_estimators=6,
    max_features=2,       
    min_sample_split=2,
    max_depth=5,
    mode="gini",
    seed=42
)

model_rf.fit(train_set)
predictions=model_rf.predict(test_set)
print('Predictions:',predictions)
print('Score:',model_rf.score(test_set))


Predictions: ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor']
Score: 0.9795918367346939
